In [1]:
DF = r'../data/processed/clusters_skills_df.pkl'
CLUSTERS_SKILLS = r'..\data\processed\clusters_skills.yaml'
MLFLOW_TRACKING_URI = '../models/mlruns'
MLFLOW_EXPERIMENT_NAME = "rf_job_predict"
EXPERIMENT_ID = 'c410d8ab2d42489ba0b8738b000d13e5'
ARTIFACT_PATH = '../models/MODELS'

In [2]:
import mlflow
import mlflow.sklearn
import os
import pickle
import yaml
import pandas as pd
import numpy as np

In [3]:
df = pd.read_pickle(DF)

In [5]:
x = df.droplevel(0, axis=1)

In [6]:
x =x.iloc[:,25:]

In [7]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)


In [8]:
target_path = ARTIFACT_PATH + '/' + EXPERIMENT_ID + '/artifacts'

In [9]:
target_path

'../models/MODELS/c410d8ab2d42489ba0b8738b000d13e5/artifacts'

In [10]:
# Get a list of folders in the specified path
folders = [entry.name for entry in os.scandir(target_path) if entry.is_dir()]

# Print the list of folders
print(folders[0])

 Basic RandomForestClassifier with PCA


In [11]:
target_path += '/' + folders[0]+ '/model.pkl'
target_path

'../models/MODELS/c410d8ab2d42489ba0b8738b000d13e5/artifacts/ Basic RandomForestClassifier with PCA/model.pkl'

In [12]:
with open(target_path, 'rb') as file:
    loaded_model = pickle.load(file)

In [13]:
loaded_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', RobustScaler(),
                                                  Index(['skills_group_0', 'skills_group_1', 'skills_group_10',
       'skills_group_11', 'skills_group_12', 'skills_group_13',
       'skills_group_14', 'skills_group_15', 'skills_group_16',
       'skills_group_17', 'skills_group_18', 'skills_group_19',
       'skills_group_2', 'skills_group_20', 'skills_group_21',
       'skills_group_22', 'skills_group_23', 'skills_group_24',
       'skills_group_3', 'skills_group_4', 'skills_group_5', 'skills_group_6',
       'skills_group_7', 'skills_group_9'],
      dtype='object'))])),
                ('pca', PCA(n_components=0.99)),
                ('classifier',
                 RandomForestClassifier(n_jobs=-1, random_state=42,
                                        verbose=1))])

In [14]:
with open(CLUSTERS_SKILLS, 'r') as file :
    clusters_skills = yaml.load(file, Loader=yaml.FullLoader)
    

In [15]:
del clusters_skills['skills_group_8']

In [16]:
# clusters_skills = pd.Series(clusters_skills)

In [17]:
# clusters_skills = clusters_skills.reset_index()

In [18]:
# clusters_skills.columns = ['Cluster', 'Skills']

In [56]:
sample_skills =  ['JavaScript', 'Python', 'Angular', 'MySQL', 'Selenium', 'Appium']
target_class = "Engineer, data"
target_score = 0

In [38]:
test = pd.DataFrame(columns=list(clusters_skills.keys()) + x.columns.to_list())

In [39]:
test.loc[0] = 0

In [40]:
for cluster, skills in clusters_skills.items() :
    for skill in skills :
        if skill in sample_skills:
            test[cluster] +=1

In [41]:
for skill in sample_skills :
    if skill in test.columns.tolist() :
        test[skill] += 1

In [42]:
test.sum()

skills_group_0     0
skills_group_1     0
skills_group_10    0
skills_group_11    0
skills_group_12    0
                  ..
liblittletest      0
npm                0
pnpm               0
snitch             0
tunit              0
Length: 260, dtype: int64

In [43]:
class_names = loaded_model.classes_
class_names

array(['Academic researcher', 'Cloud infrastructure engineer',
       'Data or business analyst',
       'Data scientist or machine learning specialist',
       'Database administrator', 'DevOps specialist',
       'Developer, QA or test', 'Developer, back-end',
       'Developer, desktop or enterprise applications',
       'Developer, front-end', 'Developer, full-stack',
       'Developer, game or graphics', 'Developer, mobile',
       'Engineer, data', 'Scientist', 'Security professional',
       'System administrator'], dtype=object)

In [44]:
predictions = loaded_model.predict_proba(test)


[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished


In [57]:
for class_name, probability in zip(class_names, predictions[0]):  # Assuming predictions has shape (1, n_classes)
    print(f"Class: {class_name}, Probability: {probability:.4f}")
    if class_name == target_class :
        target_score = probability
        

Class: Academic researcher, Probability: 0.1475
Class: Cloud infrastructure engineer, Probability: 0.0100
Class: Data or business analyst, Probability: 0.0275
Class: Data scientist or machine learning specialist, Probability: 0.0200
Class: Database administrator, Probability: 0.0075
Class: DevOps specialist, Probability: 0.0500
Class: Developer, QA or test, Probability: 0.0325
Class: Developer, back-end, Probability: 0.0900
Class: Developer, desktop or enterprise applications, Probability: 0.0525
Class: Developer, front-end, Probability: 0.1600
Class: Developer, full-stack, Probability: 0.2400
Class: Developer, game or graphics, Probability: 0.0200
Class: Developer, mobile, Probability: 0.0500
Class: Engineer, data, Probability: 0.0200
Class: Scientist, Probability: 0.0200
Class: Security professional, Probability: 0.0000
Class: System administrator, Probability: 0.0525


In [58]:
sorted_classes = sorted(zip(class_names, predictions[0]), key=lambda x: x[1], reverse=True)
for class_name, probability in sorted_classes:
    print(f"Class: {class_name}, {probability:.4f}")

Class: Developer, full-stack, 0.2400
Class: Developer, front-end, 0.1600
Class: Academic researcher, 0.1475
Class: Developer, back-end, 0.0900
Class: Developer, desktop or enterprise applications, 0.0525
Class: System administrator, 0.0525
Class: DevOps specialist, 0.0500
Class: Developer, mobile, 0.0500
Class: Developer, QA or test, 0.0325
Class: Data or business analyst, 0.0275
Class: Data scientist or machine learning specialist, 0.0200
Class: Developer, game or graphics, 0.0200
Class: Engineer, data, 0.0200
Class: Scientist, 0.0200
Class: Cloud infrastructure engineer, 0.0100
Class: Database administrator, 0.0075
Class: Security professional, 0.0000


In [47]:
idx = np.argmax(predictions)
class_names[idx]

'Developer, full-stack'

In [55]:
sorted_classes

[('Developer, full-stack', 0.2758333333333333),
 ('Academic researcher', 0.23085714285714284),
 ('Developer, front-end', 0.125),
 ('Developer, back-end', 0.10999999999999999),
 ('Developer, mobile', 0.08),
 ('System administrator', 0.0625),
 ('Developer, desktop or enterprise applications', 0.052000000000000005),
 ('Data or business analyst', 0.03),
 ('Data scientist or machine learning specialist', 0.01),
 ('DevOps specialist', 0.01),
 ('Engineer, data', 0.007142857142857143),
 ('Developer, QA or test', 0.006666666666666666),
 ('Cloud infrastructure engineer', 0.0),
 ('Database administrator', 0.0),
 ('Developer, game or graphics', 0.0),
 ('Scientist', 0.0),
 ('Security professional', 0.0)]

In [48]:
d = {}

In [49]:
others = []
for skill in x.columns.tolist() :
    if skill not in sample_skills:
        others.append(skill)

In [64]:
for skill in others :
    new = test.copy()
    for cluster, skills in clusters_skills.items() :
        if skill in skills:
            new[cluster] +=1
    predictions = loaded_model.predict_proba(new)
    target_class_index = list(class_names).index(target_class)
    target_class_score = predictions[0][target_class_index]
    target_class_score = (target_class_score - target_score) / target_score
    d[skill] = target_class_score
    

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0

[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurr

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0

[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurr

In [65]:
sorted_items = sorted(d.items(), key=lambda x: x[1], reverse=True)


In [68]:
threshold = 1
suggestions = []
for key, value in sorted_items:
    if value > threshold:
        suggestions.append(key)
    print(f"{key}: {value}")

Scala: 2.056547619047619
Cassandra: 2.056547619047619
Snowflake: 2.056547619047619
Play Framework: 2.056547619047619
Apache Kafka: 2.056547619047619
Apache Spark: 2.056547619047619
Hadoop: 2.056547619047619
Cloudflare: 1.375
Digital Ocean: 1.375
Hetzner: 1.375
Linode, now Akamai: 1.375
Vultr: 1.375
R: 0.5697055137844612
Django: 0.5697055137844612
FastAPI: 0.5697055137844612
Flask: 0.5697055137844612
CUDA: 0.5697055137844612
Hugging Face Transformers: 0.5697055137844612
Keras: 0.5697055137844612
NumPy: 0.5697055137844612
Opencv: 0.5697055137844612
Pandas: 0.5697055137844612
Scikit-Learn: 0.5697055137844612
TensorFlow: 0.5697055137844612
Tidyverse: 0.5697055137844612
Torch/PyTorch: 0.5697055137844612
Pip: 0.5697055137844612
GDScript: 0.4999999999999999
Go: 0.4999999999999999
Lua: 0.4999999999999999
Rust: 0.4999999999999999
Dynamodb: 0.4999999999999999
Elasticsearch: 0.4999999999999999
InfluxDB: 0.4999999999999999
Neo4J: 0.4999999999999999
PostgreSQL: 0.4999999999999999
Redis: 0.499999999

In [73]:
print ('Your skill set is {}:'.format(sample_skills))
print ('============================')
print ('Your Target Role is {}:'.format(target_class))
print ('============================')
print ('You may consider learning {}:'.format(suggestions))


Your skill set is ['JavaScript', 'Python', 'Angular', 'MySQL', 'Selenium', 'Appium']:
Your Target Role is Engineer, data:
You may consider learning ['Scala', 'Cassandra', 'Snowflake', 'Play Framework', 'Apache Kafka', 'Apache Spark', 'Hadoop', 'Cloudflare', 'Digital Ocean', 'Hetzner', 'Linode, now Akamai', 'Vultr']:


In [71]:
print (sample_skills)

['JavaScript', 'Python', 'Angular', 'MySQL', 'Selenium', 'Appium']
